rename the downloaded images

In [17]:
import os
import uuid
import re

base_dir = root if isinstance(globals().get("root", None), str) else "dataset/"

def original_basename(name: str) -> str:
    # If name is a tmp name like "__tmp__<uuid>__<orig>", sort by the original part
    return name.rsplit("__", 1)[-1] if name.startswith("__tmp__") else name

def natural_key(s: str):
    # Case-insensitive natural sort (numbers in names sorted numerically, like macOS)
    return [int(t) if t.isdigit() else t.casefold() for t in re.split(r"(\d+)", s)]

# Collect non-hidden files directly under base_dir
names = [
    n for n in os.listdir(base_dir)
    if not n.startswith(".") and os.path.isfile(os.path.join(base_dir, n))
]

# Sort like macOS Finder (case-insensitive natural)
names.sort(key=lambda n: natural_key(original_basename(n)))

# First, move every file to a unique tmp name to avoid collisions
tmp_paths = []
for name in names:
    src = os.path.join(base_dir, name)
    if name.startswith("__tmp__"):
        tmp_paths.append(src)
        continue
    tmp_name = f"__tmp__{uuid.uuid4().hex}__{name}"
    tmp_path = os.path.join(base_dir, tmp_name)
    os.rename(src, tmp_path)
    tmp_paths.append(tmp_path)

# Then rename in pairs: on_model{1}, still-life{1}, on_model{2}, still-life{2}, ...
for i, tmp_path in enumerate(tmp_paths):
    pair_idx = i // 2 + 1
    role = "on_model" if i % 2 == 0 else "still-life"
    _, ext = os.path.splitext(tmp_path)
    target = os.path.join(base_dir, f"{role}{pair_idx}{ext.lower()}")
    os.rename(tmp_path, target)


pairing

In [ ]:
# Stitch pairs: left from gt/still-life{n}.jpg, right from degraded/on_model{n}.jpg
import os

gt_dir = "gt/"
ext = extname.lower() if isinstance(globals().get("extname", None), str) else ".jpg"

prep_dir = "pairs/control/"
os.makedirs(prep_dir, exist_ok=True)

pattern = re.compile(rf"^still-life(\d+){re.escape(ext)}$", re.IGNORECASE)

saved = 0
for fname in os.listdir(gt_dir):
    m = pattern.match(fname)
    if not m:
        continue
    idx = int(m.group(1))
    left_path = os.path.join(gt_dir, fname)
    right_path = os.path.join(base_dir, f"on_model{idx}{ext}")

    if not os.path.isfile(right_path):
        continue

    try:
        left = Image.open(left_path).convert("RGB")
        right = Image.open(right_path).convert("RGB")

        w1, h1 = left.size
        w2, h2 = right.size
        H = max(h1, h2)
        W = w1 + w2

        canvas = Image.new("RGB", (W, H), (255, 255, 255))
        top1 = (H - h1) // 2
        top2 = (H - h2) // 2

        canvas.paste(left, (0, top1))
        canvas.paste(right, (w1, top2))

        out_path = os.path.join(prep_dir, f"pair{idx}{ext}")
        save_kwargs = {}
        if ext in (".jpg", ".jpeg"):
            save_kwargs.update(quality=95, optimize=True)
        canvas.save(out_path, **save_kwargs)
        saved += 1
    except Exception as e:
        print(f"Failed to stitch index {idx}: {e}")

print(f"Stitched and saved {saved} pairs to {prep_dir}")

In [ ]:
# Stitch training pairs: left from gt/still-life{n}.jpg, right from gt/on_model{n}.jpg
from PIL import Image
import os
import re

prep_dir = "training_pairs/gt/"
os.makedirs(prep_dir, exist_ok=True)

pattern = re.compile(rf"^still-life(\d+){re.escape(ext)}$", re.IGNORECASE)

saved = 0
for fname in os.listdir(gt_dir):
    m = pattern.match(fname)
    if not m:
        continue
    idx = int(m.group(1))
    left_path = os.path.join(gt_dir, f"still-life{idx}{ext}")
    right_path = os.path.join(gt_dir, f"on_model{idx}{ext}")

    if not (os.path.isfile(left_path) and os.path.isfile(right_path)):
        continue

    try:
        left = Image.open(left_path).convert("RGB")
        right = Image.open(right_path).convert("RGB")

        w1, h1 = left.size
        w2, h2 = right.size
        H = max(h1, h2)
        W = w1 + w2

        canvas = Image.new("RGB", (W, H), (255, 255, 255))
        top1 = (H - h1) // 2
        top2 = (H - h2) // 2

        canvas.paste(left, (0, top1))
        canvas.paste(right, (w1, top2))

        out_path = os.path.join(prep_dir, f"pair{idx}{ext}")
        save_kwargs = {}
        if ext in (".jpg", ".jpeg"):
            save_kwargs.update(quality=95, optimize=True)
        canvas.save(out_path, **save_kwargs)
        saved += 1
    except Exception as e:
        print(f"Failed to stitch index {idx}: {e}")

print(f"Stitched and saved {saved} pairs to {prep_dir}")


Stitched and saved 60 pairs to pairs


resize pairs to match one of Kontext res:

672×1568, 688×1504, 720×1456, 752×1392, 800×1328, 832×1248, 880×1184, 944×1104, 1024×1024, 1104×944, 1184×880, 1248×832, 1328×800, 1392×752, 1456×720, 1504×688, 1568×672. 



In [2]:
from PIL import Image
import os

base_dir = "/Users/lucamartini/Desktop/target/"
target_size = (1328, 800)  # (target_w, target_h)
processed = 0

target_w, target_h = target_size

for fname in os.listdir(base_dir):
    if fname.startswith(".") or not fname.lower().endswith(".jpg"):
        continue

    fpath = os.path.join(base_dir, fname)
    if not os.path.isfile(fpath):
        continue

    try:
        with Image.open(fpath) as im:
            # Preserve aspect ratio: fix height to target_h
            w, h = im.size
            if h == 0:
                continue
            new_w = max(1, int(round(w * (target_h / h))))
            resized = im.resize((new_w, target_h), resample=Image.LANCZOS)

            # Ensure proper mode for JPEG
            if resized.mode not in ("RGB", "L"):
                resized = resized.convert("RGB")

            # If the target width is larger than resized width, pad on the right
            if new_w < target_w:
                # Average color of the rightmost column as "border color"
                right_col = resized.crop((new_w - 1, 0, new_w, target_h)).resize((1, 1), Image.BOX)
                border_color = right_col.getpixel((0, 0))

                canvas = Image.new("RGB", (target_w, target_h), border_color)
                canvas.paste(resized, (0, 0))
                out_img = canvas
            else:
                out_img = resized  # wider than target; leave as-is (no padding)

            # JPEG save options
            save_kwargs = {}
            if fname.lower().endswith((".jpg", ".jpeg")):
                save_kwargs.update(quality=95, optimize=True)

            out_img.save(fpath, **save_kwargs)
            processed += 1
    except Exception:
        # Skip non-images or unreadable files
        pass

print(f"Processed {processed} images in {base_dir}; height={target_h}, padded right to width {target_w} when needed")

Processed 5 images in /Users/lucamartini/Desktop/target/; height=800, padded right to width 1328 when needed


In [8]:
# mask with split shifted 15px to the left (right white side larger)
src_dir = "training_pairs/gt/"
dst_dir = "training_pairs/masks/"
os.makedirs(dst_dir, exist_ok=True)

offset_px = 15  # shift split left by 15px
created = 0
for fname in os.listdir(src_dir):
    if fname.startswith("."):
        continue
    src_path = os.path.join(src_dir, fname)
    if not os.path.isfile(src_path):
        continue
    try:
        with Image.open(src_path) as im:
            w, h = im.size
            mask = Image.new("L", (w, h), 255)  # right side = white (255)

            left_w = max(0, min(w, w // 2 - offset_px))  # shrink left (black) by 15px
            if left_w > 0:
                left_half = Image.new("L", (left_w, h), 0)  # left side = black (0)
                mask.paste(left_half, (0, 0))

            out_name = os.path.splitext(fname)[0] + ".png"
            out_path = os.path.join(dst_dir, out_name)
            mask.save(out_path)
            created += 1
    except Exception:
        pass

print(f"Created {created} masks with split {offset_px}px left of center in {dst_dir}")

Created 60 masks with split 15px left of center in training_pairs/masks/


In [12]:
from PIL import Image

# Stitch two images side by side with no stretching (centered vertically)

def stitch_images(left_path, right_path, out_path):
    left = Image.open(left_path).convert("RGB")
    right = Image.open(right_path).convert("RGB")

    w1, h1 = left.size
    w2, h2 = right.size
    H = max(h1, h2)
    W = w1 + w2

    canvas = Image.new("RGB", (W, H), (255, 255, 255))
    top1 = (H - h1) // 2
    top2 = (H - h2) // 2

    canvas.paste(left, (0, top1))
    canvas.paste(right, (w1, top2))
    canvas.save(out_path)

stitch_images("/Users/lucamartini/Desktop/Shootify ML Engineer Coding challenge dataset/05_still-life.jpg", 
              "/Users/lucamartini/Desktop/Shootify ML Engineer Coding challenge dataset/05_on-model.jpg", 
              "/Users/lucamartini/Desktop/05_control.jpg")